In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
# import os
# os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/Kaggle3"

In [ ]:
# %cd /content/drive/MyDrive/Kaggle3
# # !kaggle datasets download -d grassknoted/asl-alphabet
# # #!kaggle datasets download -d risangbaskoro/wlasl-processed
# !kaggle datasets download -d lexset/synthetic-asl-alphabet #

In [ ]:
# !unzip \*.zip # unzipping the datasets

In [4]:
import os
import cv2
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, TimeDistributed, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image


In [ ]:
import os
train_dir = '/content/drive/MyDrive/Kaggle3/Train_Alphabet/'
categories = os.listdir(train_dir)
len(categories)
# length of training images
length = 0
train_images_names_and_paths = {}

for cat in categories:
    train_images_names_and_paths[cat] = os.listdir(train_dir + '/' + cat)
    length += len(os.listdir(train_dir + '/' + cat))

print('Total Length of Training Images:', length)

Total Length of Training Images: 24300


In [ ]:
# Step 1: Load ASL Alphabet dataset
train_data_dir = '/content/drive/MyDrive/Kaggle3/Train_Alphabet/'
test_data_dir = '/content/drive/MyDrive/Kaggle3/Test_Alphabet/'

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.applications import VGG19
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
import seaborn as sns
from keras.preprocessing.image import img_to_array, array_to_img
from keras.applications.vgg19 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator



In [ ]:

train_data_dir = '/content/drive/MyDrive/Kaggle3/Train_Alphabet/'
test_data_dir = '/content/drive/MyDrive/Kaggle3/Test_Alphabet/'

# Initialize ImageDataGenerator for train and test data
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 32 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(48, 48),
        batch_size=32,
        class_mode='categorical')

# Flow validation images in batches of 32 using test_datagen generator
test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(48, 48),
        batch_size=32,
        class_mode='categorical')

# Now you can use train_generator and test_generator for training and testing your model


Found 24300 images belonging to 27 classes.
Found 2700 images belonging to 27 classes.


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Initialize a Sequential model
model = Sequential()

# Add convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flatten the output for Dense layers
model.add(Flatten())

# Add Dense layers
model.add(Dense(128, activation='relu'))
model.add(Dense(27, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
      train_generator,
      steps_per_epoch=train_generator.samples // train_generator.batch_size,
      # epochs=10,
      epochs=1,
      validation_data=test_generator,
      validation_steps=test_generator.samples // test_generator.batch_size)


NameError: name 'train_generator' is not defined

In [ ]:
# Step 10: Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

85/85 [==============================] - 34s 397ms/step - loss: 0.4954 - accuracy: 0.8478
Test Loss: 0.49539750814437866, Test Accuracy: 0.847777783870697


In [ ]:
predictions = model.predict(test_generator)

85/85 [==============================] - 34s 394ms/step


In [ ]:
# # create labels.txt

# # Get class names from the dataset
# class_names = train_datagen.class_names

# # Save class names to a text file
# with open('labels0.txt', 'w') as f:
#     for class_name in class_names:
#         f.write(class_name + '\n')


SAVE MODEL

OPTION 1

In [31]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
import keras


In [32]:
from keras.models import load_model

keras_file = '/content/drive/MyDrive/mobilenet_model.h5'
model = load_model(keras_file)


In [ ]:
import tensorflow as tf

# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)



OPTION2

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import save_model


In [ ]:
model.save('detect.keras')

convert to tensorflow lite

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)


In [ ]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

In [ ]:
model_tflite = converter.convert()

In [ ]:
open( "detect.tflite", "wb").write(model_tflite)

369384

QUANTIZED

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
model_tflite = converter.convert()
open( "Quantized_detect.tflite", "wb").write(model_tflite)

723632

INCLUDE METADATA

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

metadata = model.signatures['serving_default'].metadata
# Access the metadata

# Extract the labels
labels = metadata['labels']

# Now you can use the labels as needed
print(labels)


EXPLAINABILITY